In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import glob
import argparse
import random
import numpy as np
import lmdb
import cv2
import pickle
import caffe
from caffe.proto import caffe_pb2
from time import time
from sklearn.model_selection import train_test_split

In [5]:
!apt-get install caffe-cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  caffe-tools-cpu cython3 fonts-lyx javascript-common libblosc1 libcaffe-cpu1
  libgflags2.2 libgoogle-glog0v5 libjs-jquery-ui libleveldb1v5 liblmdb0
  python-matplotlib-data python-tables-data python3-bs4 python3-caffe-cpu
  python3-chardet python3-cycler python3-dateutil python3-decorator
  python3-gflags python3-h5py python3-html5lib python3-ipython
  python3-ipython-genutils python3-leveldb python3-lxml python3-matplotlib
  python3-networkx python3-nose python3-numexpr python3-olefile python3-pandas
  python3-pandas-lib python3-pexpect python3-pickleshare python3-pil
  python3-pkg-resources python3-prompt-toolkit python3-protobuf
  python3-ptyprocess python3-pygments python3-pyparsing python3-pywt
  python3-scipy python3-simplegeneric python3-six python3-skimage
  python3-skimage-lib python3-tables python3-tables-lib python3-traitlets


## Directory parser for file .py

In [0]:
# ap = argparse.ArgumentParser()
# ap.add_argument('-p', '--path', required=True, help='Path to dataset directory')
# args = vars(ap.parse_args())

## Default imagem size for resnet architecture 

In [0]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

In [0]:
def transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT, equalize=False):
  """Function that resize an image and equalize it if necessary."""
  if equalize:
    # Histogram Equalization
    img[:, :, 0] = cv2.equalizeHist(img[:, :, 0])
    img[:, :, 1] = cv2.equalizeHist(img[:, :, 1])
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])
    
  # Image Resizing
  img = cv2.resize(img, (img_width, img_height), interpolation=cv2.INTER_CUBIC)
  
  return img

In [0]:
def make_datum(img, label):
  # Image is numpy.ndarray format. BGR instead of RGB
  return caffe_pb2.Datum(channels=3,
                         width=IMAGE_WIDTH,
                         height=IMAGE_HEIGHT,
                         label=label,
                         data=np.rollaxis(img, 2).tostring())

## Creating train and validation datasets with lmdb lib

In [0]:
path = '/////ABS/PATH'
parent_path = os.path.sep.join(path.split(os.path.sep)[:-1])
sibling_path = path.split(os.path.sep)[-1] + '_lmdb'
sibling_path = os.path.sep.join([parent_path, sibling_path])
train_lmdb = os.path.sep.join([sibling_path, 'train'])
validation_lmdb = os.path.sep.join([sibling_path, 'validation'])

In [0]:
if not os.path.exists(sibling_path):
  os.makedirs(sibling_path)

In [14]:
os.system('rm -rf ' + train_lmdb)
os.system('rm -rf ' + validation_lmdb)

0

In [0]:
dataset = []
for r, dirs, files in os.walk(path):
  if len(dirs) > 0:
    labels = dirs
    continue # use only leaf folders
  files_full_path = ['{}/{}'.format(r, f) for f in files]
  directory_name = r.split(os.path.sep)[-1]
  dataset.append((files_full_path, directory_name))

In [0]:
label_dict = [(l, i) for i, l in enumerate(labels)]
label_dict = dict(label_dict)

'''
Save dictionary in the form of:

label_dict = { \
'basalcellcarcinoma': 0, \
'lentigo': 1, \
'malignantmelanoma': 2, \
'pigmentednevus': 3, \
'seborrheickeratosis': 4, \
'wart': 5, \
... \
}
'''

In [0]:
with open('label_dict.pkl', 'wb') as f:
  pickle.dump(label_dict, f)
  f.close()

In [0]:
X = [(img, label) for ndataset, label in dataset for img in ndataset]
y = [label_dict[label] for _, label in X]

In [0]:
random.shuffle(X)

## Spliting datasets

In [0]:
train_data, test_data, _, _ = train_test_split(X, y, train_size=0.8, stratify=y)

In [27]:
print('Creating train_lmdb...')

Creating train_lmdb...


In [28]:
train_time = time()
in_db = lmdb.open(train_lmdb, map_size=int(1e12))
with in_db.begin(write=True) as in_txn:
  for in_idx, (img_path, label) in enumerate(train_data):
    if in_idx % 100 == 0:
      print('Processed {}/{}'.format(in_idx, len(train_data)), end='\r')

  img = cv2.imread(img_path, cv2.IMREAD_COLOR)
  img = transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT)
 
  num_label = label_dict[label]
  datum = make_datum(img, num_label)

  key = '{:0>6d}'.format(in_idx)
  in_txn.put(key.encode(), datum.SerializeToString())

NameError: ignored

In [0]:
in_db.close()
print('Finished {} train_lmdb in {:.2f} sec'.format(len(train_data), (time() - train_time)))

In [0]:
print('\nCreating validation_lmdb...')

In [0]:
test_time = time()
in_db = lmdb.open(validation_lmdb, map_size=int(1e12))
with in_db.begin(write=True) as in_txn:
  old_t = time()
  for in_idx, (img_path, label) in enumerate(test_data):
    if in_idx % 100 == 0:
      print('Processed {}/{}'.format(in_idx, len(test_data)), end='\r\r')
      
      old_t = time()
      
  img = cv2.imread(img_path, cv2.IMREAD_COLOR)
  img = transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT)
  num_label = label_dict[label]
  datum = make_datum(img, num_label)
  
  key = '{:0>6d}'.format(in_idx)
  in_txn.put(key.encode(), datum.SerializeToString())

In [29]:
in_db.close()
print('Finished {} test_lmdb in {:.2f} sec'.format(len(test_data), (time() - test_time)))

NameError: ignored

In [0]:
print('\nFinished processing all images in {:.2f}'.format(time() - train_time))